In [1]:
real = True
manual_today = '2023_03_19'
manual_yesterday = '2023_03_18'

In [2]:
import json
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests
import time
import os
from datetime import date

NEWSPAPER = 'CNN'
SECTIONS = ['business', 'entertainment', 'politics', 'us', 'world']
NEWSPAPER_URL = "https://www.cnn.com"

In [3]:
if real:
    today = date.today()
    today = str(today).replace('-', '_')
    print("Today's date:", today)
else:
    today = manual_today
    yesterday = manual_yesterday
    print("Today's date:", today)

Today's date: 2023_03_28


In [4]:
os.getcwd()

'C:\\Users\\danzh\\Documents\\Projects\\FiftyNines\\scrapers'

In [5]:
db = pd.read_csv('db_current.csv')

In [6]:
db.shape

(2212, 7)

In [7]:
article_body_collected = list(db['link_to_original'].values)
len(article_body_collected)

2212

In [8]:
scraped_today = ['WSJ', 'NYT', 'Fox']
for scraped_file in scraped_today:
    scraped_file_name = scraped_file + '_' + today + '.csv'
    temp_df = pd.read_csv('scrape_results\\' + today + '\\' + scraped_file_name)
    scraped_today_links = list(temp_df['link_to_original'].values)
    article_body_collected += scraped_today_links

In [9]:
article_body_collected_set = set(article_body_collected)
len(article_body_collected_set)

2330

In [10]:
article_body_collected = list(article_body_collected_set)
len(article_body_collected)

2330

In [11]:
def cnn_extract_article(content, a_a, title_div):
    a = content.find("a", {"class": a_a})
    article_link = NEWSPAPER_URL + a['href']
    if a.find("div", {"class": title_div}) is not None:
        article_title = a.find("div", {"class": title_div}).text.strip()
    else:
        article_title = article.find("div", {"class": "container__headline container_lead-plus-headlines__headline"}).text.strip()

    article_web = requests.get(article_link)
    article_content = soup(article_web.content,'html.parser')

    article_content_script = article_content.find("script", {"type": "application/ld+json"})
    article_body = json.loads(article_content_script.text)['articleBody']
    article_dt = json.loads(article_content_script.text)['dateCreated'].split('T')[0]
    article_time = int(pd.to_datetime(json.loads(article_content_script.text)['dateCreated']).timestamp()) * 1000

    return(article_link, article_title, article_body, article_dt, article_time)

In [12]:
%%time
list_article_title = []
list_article_link = []
list_article_body = []
list_article_dt = []
list_article_time = []
list_article_section = []
for section in SECTIONS:
    newspaper_section_url = NEWSPAPER_URL + "/" + section
    web = requests.get(newspaper_section_url)
    section_content = soup(web.content,'html.parser')
    if section == 'entertainment':
        section_div = 'card container__item container__item--type-section container_lead-plus-headlines-with-images__item container_lead-plus-headlines-with-images__item--type-section'
        a_a = 'container__link container_lead-plus-headlines-with-images__link container_lead-plus-headlines-with-images__left container_lead-plus-headlines-with-images__light'
        title_div = 'container__headline container_lead-plus-headlines-with-images__headline'
    else:
        section_div = 'card container__item container__item--type-section container_lead-plus-headlines__item container_lead-plus-headlines__item--type-section'
        a_a = 'container__link container_lead-plus-headlines__link'
        title_div = 'container__headline container_lead-plus-headlines__headline'
    section_relevant_content = section_content.find_all("div", {"class": section_div})
    
    attempts = 0
    for article in section_relevant_content:
        try:
            article_link, article_title, article_body, article_dt, article_time = cnn_extract_article(article, a_a, title_div)
        except:
            try:
                article_link, article_title, article_body, article_dt, article_time = cnn_extract_article(article, a_a, title_div)
            except:
                try:
                    title_div = "headline__wrapper"
                    article_link, article_title, article_body, article_dt, article_time = cnn_extract_article(article, a_a, title_div)
                except:
                    pass
        if article_link in article_body_collected:
            pass
        else:
            if article_link not in list_article_link:
                list_article_title.append(article_title)
                list_article_link.append(article_link)
                list_article_body.append(article_body)
                list_article_dt.append(article_dt)
                list_article_time.append(article_time)
                list_article_section.append(section)
            else:
                index_of_interest = list_article_link.index(article_link)
                if isinstance(list_article_section[index_of_interest], list):
                    list_article_section[index_of_interest].append(section)
                else:
                    new_list = [list_article_section[index_of_interest]]
                    new_list.append(section)
                    list_article_section[index_of_interest] = new_list

NameError: name 'article_link' is not defined

In [13]:
articles_dict = {'title':list_article_title,
    'link_to_original':list_article_link,
    'article_body':list_article_body,
    'article_dt':list_article_dt,
    'article_time':list_article_time,
    'section': list_article_section,
   }
    
df = pd.DataFrame(articles_dict)

In [18]:
filename = NEWSPAPER + '_' + today + '.csv'

In [19]:
df.to_csv('\\scrape_results\\' + today + '\\' + filename, index = False)